# Paso 1: Configuración del Entorno

In [0]:
%pip install beautifulsoup4 requests pandas

Python interpreter will be restarted.
Python interpreter will be restarted.


# Importar librerías

In [0]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

# Paso 2: Clase DataWeb

### Clase especializada para extraer datos financieros de Yahoo Finance

In [0]:




class DataWeb:
    def __init__(self):
        self.url = "https://es.finance.yahoo.com/quote/DOLA-USD/history/"
    

    def obtener_datos(self):
        try:
            # url , cabeceras
            headers = {
                'User-Agent': 'Mozilla/5.0'
            }
            respuesta = requests.get(self.url,headers=headers)
            if respuesta.status_code != 200:
                print("La url saco error, no respondio o no existe")
            #print(respuesta.text)
            soup = BeautifulSoup(respuesta.text,'html.parser')
            tabla = soup.select_one('div[data-testid="history-table"] table')
            nombre_columnas = [th.get_text(strip=True) for th in tabla.thead.find_all('th')]
            filas = []
            for tr in tabla.tbody.find_all('tr'):
                columnas = [ td.get_text(strip=True) for td in tr.find_all('td')]
                if len(columnas) == len(nombre_columnas):
                    filas.append(columnas)
            df = pd.DataFrame(filas,columns=nombre_columnas).rename(columns = {
                'Fecha': 'fecha',
                'Abrir': 'abrir',
                'Máx.': 'max',
                'Mín.': 'min',
                'CerrarPrecio de cierre ajustado para splits.': 'cerrar',
                'Cierre ajustadoPrecio de cierre ajustado para splits y distribuciones de dividendos o plusvalías.': 'cierre_ajustado',
                'Volumen':'volumen'
            })
            df = self.convertir_numericos(df)
            #df.to_excel("dataweb_limpio.xlsx")
            # print(nombre_columnas)
            # print(filas)
            # for th in tabla.thead.find_all('th'):
            #     th_data = th.get_text(strip=True)
# ['Fecha', 'Abrir', 'Máx.', 'Mín.', 'CerrarPrecio de cierre ajustado para splits.', 'Cierre ajustadoPrecio de cierre ajustado para splits y distribuciones de dividendos o plusvalías.', 'Volumen']

            return df
        except Exception as err:
            print("Error en la funcion obtener_datos")
            df = pd.DataFrame()
            return df


    def convertir_numericos(self,df=pd.DataFrame()):
        df= df.copy()
        if len(df)>0:
            #for col in (df.columns):
            for col in ('abrir',	'max',	'min',	'cerrar',	'cierre_ajustado',	'volumen'):
                df[col] = (df[col]
                           .str.replace(r"\.","",regex=True)
                           .str.replace(",",'.'))

        return df

# Paso 3: Crear Esquema y Tablas Delta

In [0]:
%sql
DROP TABLE IF EXISTS finanzas_schema.indicador_parametros;
DROP TABLE IF EXISTS finanzas_schema.indicador_financiero;
DROP TABLE IF EXISTS finanzas_schema.indicador_demografico;


In [0]:
%sql
DROP SCHEMA IF EXISTS finanzas_schema CASCADE;

In [0]:
%sql
-- Crear esquema principal
CREATE SCHEMA IF NOT EXISTS finanzas_schema
COMMENT 'Esquema para datos financieros y análisis de mercados';

In [0]:
%sql
-- TABLA 1: PARÁMETROS DE INDICADORES
-- =================================================
DROP TABLE IF EXISTS finanzas_schema.indicador_parametros;



In [0]:
%fs rm -r dbfs:/user/hive/warehouse/finanzas_schema.db/indicador_parametros

res1: Boolean = false

In [0]:
%sql
CREATE TABLE IF NOT EXISTS finanzas_schema.indicador_parametros (
    cod_indicador STRING NOT NULL,
    descripcion STRING,
    prioridad INT,
    patron_clasificacion STRING,
    activo BOOLEAN,
    fecha_creacion TIMESTAMP
)
USING PARQUET
COMMENT 'Tabla de configuración y patrones para clasificación de instrumentos financieros';

In [0]:
%sql
select * from finanzas_schema.indicador_parametros ;

cod_indicador,descripcion,prioridad,patron_clasificacion,activo,fecha_creacion


In [0]:
%sql
SHOW CREATE TABLE finanzas_schema.indicador_parametros;

createtab_stmt
"CREATE TABLE finanzas_schema.indicador_parametros ( cod_indicador STRING, descripcion STRING, prioridad INT, patron_clasificacion STRING, activo BOOLEAN, fecha_creacion TIMESTAMP) USING PARQUET COMMENT 'Tabla de configuración y patrones para clasificación de instrumentos financieros'"


In [0]:
%sql
SHOW TABLES IN finanzas_schema;

database,tableName,isTemporary
finanzas_schema,indicador_parametros,false


In [0]:
%sql
-- TABLA 1: DATOS HISTÓRICOS FINANCIEROS

CREATE TABLE IF NOT EXISTS finanzas_schema.indicador_financiero (
    fecha STRING,
    abrir DOUBLE,
    max DOUBLE,
    min DOUBLE,
    cerrar DOUBLE,
    cierre_ajustado DOUBLE,
    volumen DOUBLE,
    cod_indicador STRING,
    year INT,
    month INT,
    day INT,
    year_month STRING,
    fecha_extraccion TIMESTAMP

)
USING PARQUET
--LOCATION 'dbfs:/delta/finanzas/indicador_financiero'
PARTITIONED BY (cod_indicador)
COMMENT 'Tabla principal con datos históricos de instrumentos financieros'
;

In [0]:
%sql
SHOW TABLES IN finanzas_schema;

database,tableName,isTemporary
finanzas_schema,indicador_financiero,false
finanzas_schema,indicador_parametros,false


In [0]:
%sql
SHOW TABLES IN finanzas_schema;

database,tableName,isTemporary
finanzas_schema,indicador_financiero,false
finanzas_schema,indicador_parametros,false


In [0]:
%sql
DESCRIBE EXTENDED finanzas_schema.indicador_parametros;

col_name,data_type,comment
cod_indicador,string,null
descripcion,string,null
prioridad,int,null
patron_clasificacion,string,null
activo,boolean,null
fecha_creacion,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,finanzas_schema,


In [0]:
%sql
DESCRIBE EXTENDED finanzas_schema.indicador_financiero;


col_name,data_type,comment
fecha,string,null
abrir,double,null
max,double,null
min,double,null
cerrar,double,null
cierre_ajustado,double,null
volumen,double,null
year,int,null
month,int,null
day,int,null


### # Paso 4: Poblar Tabla de Parámetros

In [0]:
%sql
-- =================================================
-- POBLAR TABLA DE PARÁMETROS
-- =================================================
INSERT INTO finanzas_schema.indicador_parametros VALUES
-- ÍNDICES BURSÁTILES
('^STOXX50E', 'Euro Stoxx 50 - Índice de las 50 principales empresas de la eurozona', 1, 'INDICE_EUROPEO', true, current_timestamp()),
('^IBEX', 'IBEX 35 - Índice principal de la bolsa española', 1, 'INDICE_ESPAÑOL', true, current_timestamp()),
('^FCHI', 'CAC 40 - Índice de la bolsa de París', 1, 'INDICE_FRANCES', true, current_timestamp()),

-- COMMODITIES
('GC=F', 'Oro - Futuros del metal precioso', 1, 'COMMODITY_METAL', true, current_timestamp()),
('CL=F', 'Petróleo WTI - Futuros de crudo', 1, 'COMMODITY_ENERGIA', true, current_timestamp()),

-- DIVISAS
('EURUSD=X', 'Euro / Dólar estadounidense', 1, 'DIVISA_MAYOR', true, current_timestamp()),

-- CRIPTOMONEDAS
('BTC-EUR', 'Bitcoin en euros', 1, 'CRYPTO_MAYOR', true, current_timestamp()),

-- ACCIONES ESPAÑOLAS (muestra)
('SAN.MC', 'Banco Santander - Acción española', 2, 'ACCION_ESPAÑOLA_BANCARIA', true, current_timestamp()),
('TEF.MC', 'Telefónica - Acción española de telecomunicaciones', 2, 'ACCION_ESPAÑOLA_TELCO', true, current_timestamp());

In [0]:
%sql
SELECT 'Parámetros insertados:' as info, COUNT(*) as cantidad 
FROM finanzas_schema.indicador_parametros;

-- Mostrar configuración
SELECT 
    cod_indicador, 
    descripcion, 
    patron_clasificacion, 
    prioridad,
    activo
FROM finanzas_schema.indicador_parametros 
ORDER BY prioridad, cod_indicador;

cod_indicador,descripcion,patron_clasificacion,prioridad,activo
BTC-EUR,Bitcoin en euros,CRYPTO_MAYOR,1,true
CL=F,Petróleo WTI - Futuros de crudo,COMMODITY_ENERGIA,1,true
EURUSD=X,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true
GC=F,Oro - Futuros del metal precioso,COMMODITY_METAL,1,true
^FCHI,CAC 40 - Índice de la bolsa de París,INDICE_FRANCES,1,true
^IBEX,IBEX 35 - Índice principal de la bolsa española,INDICE_ESPAÑOL,1,true
^STOXX50E,Euro Stoxx 50 - Índice de las 50 principales empresas de la eurozona,INDICE_EUROPEO,1,true
SAN.MC,Banco Santander - Acción española,ACCION_ESPAÑOLA_BANCARIA,2,true
TEF.MC,Telefónica - Acción española de telecomunicaciones,ACCION_ESPAÑOLA_TELCO,2,true


# Paso 5: poblar tabla indicador_financiero

In [0]:

dataweb = DataWeb()
df = dataweb.obtener_datos()
df = dataweb.convertir_numericos(df)
df["cod_indicador"] ="EURUSD=X"
#DOLA-USD






In [0]:
display(df)

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador
5 jun 2025,0999552,0999697,0999527,0999697,0999697,250832,EURUSD=X
3 jun 2025,1000013,1000547,0999477,1000044,1000044,81484,EURUSD=X
2 jun 2025,0999914,1000013,0997629,1000013,1000013,247614,EURUSD=X
1 jun 2025,0999237,1000313,0999219,0999914,0999914,33146,EURUSD=X
31 may 2025,0999141,0999642,0999027,0999237,0999237,53641,EURUSD=X
30 may 2025,0998828,0999206,0998507,0999141,0999141,171875,EURUSD=X
29 may 2025,0999747,0999824,0998639,0998828,0998828,319873,EURUSD=X
28 may 2025,0999632,0999822,0999291,0999747,0999747,428397,EURUSD=X
27 may 2025,1000068,1000160,0999615,0999632,0999632,31042,EURUSD=X
26 may 2025,1000004,1000079,0999092,1000068,1000068,35141,EURUSD=X


In [0]:
def completar_columnas(df, columna_fecha):
      try:
          # Extraer componentes y convertirlos explícitamente a enteros
          df['year'] = df['fecha_dt'].dt.year.astype('Int64')  # Tipo Int64 de pandas maneja NaN
          df['month'] = df['fecha_dt'].dt.month.astype('Int64')
          df['day'] = df['fecha_dt'].dt.day.astype('Int64')
          
          # Agregar columna year_month en formato yyyy-mm
          df['year_month'] = df['fecha_dt'].dt.strftime('%Y-%m')
          
          # Eliminar columna temporal
          df.drop('fecha_dt', axis=1, inplace=True)
          return df
          
      except Exception as e:
          # En caso de error, intentamos al menos devolver el DataFrame sin modificar
          if 'fecha_dt' in df.columns:
              df.drop('fecha_dt', axis=1, inplace=True)
          return df
  
def formatear_fechas(df, columna_fecha='fecha'):
      """
      Convierte el formato de fecha de '1 abr 2004' o '07 may 2025' a 'yyyy-mm-dd' 
      y agrega columnas year, month, day
      
      Parámetros:
      df (pandas.DataFrame): DataFrame que contiene la columna de fechas
      columna_fecha (str): Nombre de la columna que contiene las fechas
      
      Retorna:
      pandas.DataFrame: DataFrame actualizado
      """
      try:
          df_resultado = df.copy()
          
          # Diccionario de mapeo de meses en español a números
          meses = {
              'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may': '05', 'jun': '06',
              'jul': '07', 'ago': '08', 'sept': '09', 'oct': '10', 'nov': '11', 'dic': '12'
          }
          
          def convertir_fecha(fecha_str):
              if pd.isna(fecha_str) or not isinstance(fecha_str, str):
                  return None
                  
              # Limpiar la fecha (eliminar comillas y espacios adicionales)
              fecha_str = fecha_str.strip('"\'').strip()
              
              # Extraer partes de la fecha
              partes = fecha_str.split()
              if len(partes) != 3:
                  return None
                  
              dia, mes_abr, año = partes
              mes_abr = mes_abr.lower()
              
              # Verificar si el mes está en nuestro diccionario
              if mes_abr in meses:
                  mes = meses[mes_abr]
                  # Formatear día con ceros a la izquierda si es necesario
                  dia = dia.zfill(2)
                  # Construir fecha en formato ISO (yyyy-mm-dd)
                  return f"{año}-{mes}-{dia}"
              
              return None
          
          # Aplicar la función de conversión a la columna de fechas
          df_resultado[columna_fecha] = df_resultado[columna_fecha].apply(convertir_fecha)
          
          # Convertir a datetime para extraer componentes
          df_resultado['fecha_dt'] = pd.to_datetime(df_resultado[columna_fecha], errors='coerce')
          
          # Completar columnas year, month y day
          return completar_columnas(df_resultado, columna_fecha)
          
      except Exception as e:
          print("error {}".format(e))
          return df

In [0]:
display(df.columns)

Index(['fecha', 'abrir', 'max', 'min', 'cerrar', 'cierre_ajustado', 'volumen',
       'cod_indicador'],
      dtype='object')

In [0]:
df_2 = formatear_fechas(df,"fecha")
display(df_2)

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador,year,month,day,year_month
2025-06-05,0999552,0999697,0999527,0999697,0999697,250832,EURUSD=X,2025,6,5,2025-06
2025-06-03,1000013,1000547,0999477,1000044,1000044,81484,EURUSD=X,2025,6,3,2025-06
2025-06-02,0999914,1000013,0997629,1000013,1000013,247614,EURUSD=X,2025,6,2,2025-06
2025-06-01,0999237,1000313,0999219,0999914,0999914,33146,EURUSD=X,2025,6,1,2025-06
2025-05-31,0999141,0999642,0999027,0999237,0999237,53641,EURUSD=X,2025,5,31,2025-05
2025-05-30,0998828,0999206,0998507,0999141,0999141,171875,EURUSD=X,2025,5,30,2025-05
2025-05-29,0999747,0999824,0998639,0998828,0998828,319873,EURUSD=X,2025,5,29,2025-05
2025-05-28,0999632,0999822,0999291,0999747,0999747,428397,EURUSD=X,2025,5,28,2025-05
2025-05-27,1000068,1000160,0999615,0999632,0999632,31042,EURUSD=X,2025,5,27,2025-05
2025-05-26,1000004,1000079,0999092,1000068,1000068,35141,EURUSD=X,2025,5,26,2025-05


In [0]:
def convertir_columnas_numericas_pandas(df, columnas_string=None):
 
    # Crear una copia del DataFrame para evitar modificar el original
    df_modificado = df.copy()
    
    # Convertir columnas a string
    if columnas_string:
        for columna in columnas_string:
            if columna in df.columns:
                df_modificado[columna] = df_modificado[columna].fillna(0).astype('int64')
    return df_modificado

In [0]:
columnas= ['abrir', 'max', 'min', 'cerrar', 'cierre_ajustado', 'volumen']
df_2 = convertir_columnas_numericas_pandas(df_2,columnas)
display(df_2.head())

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador,year,month,day,year_month
2025-06-05,999552,999697,999527,999697,999697,250832,EURUSD=X,2025,6,5,2025-06
2025-06-03,1000013,1000547,999477,1000044,1000044,81484,EURUSD=X,2025,6,3,2025-06
2025-06-02,999914,1000013,997629,1000013,1000013,247614,EURUSD=X,2025,6,2,2025-06
2025-06-01,999237,1000313,999219,999914,999914,33146,EURUSD=X,2025,6,1,2025-06
2025-05-31,999141,999642,999027,999237,999237,53641,EURUSD=X,2025,5,31,2025-05


### Esquema y estructura de la tabla 2 (tabla web)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType, IntegerType

schema_fina = StructType([
    StructField("fecha", StringType(), False, {'comment': 'Fecha de la cotización en formato YYYY-MM-DD'}),
    StructField("abrir", DoubleType(), True, {'comment': 'Precio de apertura del día'}),
    StructField("max", DoubleType(), True, {'comment': 'Precio máximo alcanzado en el día'}),
    StructField("min", DoubleType(), True, {'comment': 'Precio mínimo alcanzado en el día'}),
    StructField("cerrar", DoubleType(), True, {'comment': 'Precio de cierre del día'}),
    StructField("cierre_ajustado", DoubleType(), True, {'comment': 'Precio de cierre ajustado por dividendos y splits'}),
    StructField("volumen", DoubleType(), True, {'comment': 'Volumen de acciones negociadas'}),
    StructField("cod_indicador", StringType(), True, {'comment': 'Código del activo financiero'}),
    StructField("year", IntegerType(), True, {'comment': 'Fecha de la cotización en formato YYYY'}),
    StructField("month", IntegerType(), True, {'comment': 'Fecha de la cotización en formato MM'}),
    StructField("day", IntegerType(), True, {'comment': 'Fecha de la cotización en formato DD'}),
    StructField("year_month", StringType(), True, {'comment': 'Fecha de la cotización en formato YYYYMM'})
    #,StructField("fecha_extraccion", TimestampType(), True, {'comment': 'Timestamp de cuando se extrajo la información'})
])


### Conertir Dataframe pandas a df Spark

In [0]:
from pyspark.sql.functions import lit, current_timestamp
df_spark = spark.createDataFrame(df_2,schema=schema_fina)
spark_df = df_spark.withColumn("fecha_extraccion", current_timestamp())
display(spark_df)

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador,year,month,day,year_month,fecha_extraccion
2025-06-05,999552.0,999697.0,999527.0,999697.0,999697.0,250832.0,EURUSD=X,2025,6,5,2025-06,2025-06-05T01:24:09.947+0000
2025-06-03,1000013.0,1000547.0,999477.0,1000044.0,1000044.0,81484.0,EURUSD=X,2025,6,3,2025-06,2025-06-05T01:24:09.947+0000
2025-06-02,999914.0,1000013.0,997629.0,1000013.0,1000013.0,247614.0,EURUSD=X,2025,6,2,2025-06,2025-06-05T01:24:09.947+0000
2025-06-01,999237.0,1000313.0,999219.0,999914.0,999914.0,33146.0,EURUSD=X,2025,6,1,2025-06,2025-06-05T01:24:09.947+0000
2025-05-31,999141.0,999642.0,999027.0,999237.0,999237.0,53641.0,EURUSD=X,2025,5,31,2025-05,2025-06-05T01:24:09.947+0000
2025-05-30,998828.0,999206.0,998507.0,999141.0,999141.0,171875.0,EURUSD=X,2025,5,30,2025-05,2025-06-05T01:24:09.947+0000
2025-05-29,999747.0,999824.0,998639.0,998828.0,998828.0,319873.0,EURUSD=X,2025,5,29,2025-05,2025-06-05T01:24:09.947+0000
2025-05-28,999632.0,999822.0,999291.0,999747.0,999747.0,428397.0,EURUSD=X,2025,5,28,2025-05,2025-06-05T01:24:09.947+0000
2025-05-27,1000068.0,1000160.0,999615.0,999632.0,999632.0,31042.0,EURUSD=X,2025,5,27,2025-05,2025-06-05T01:24:09.947+0000
2025-05-26,1000004.0,1000079.0,999092.0,1000068.0,1000068.0,35141.0,EURUSD=X,2025,5,26,2025-05,2025-06-05T01:24:09.947+0000


### Insertar los datos en la tabla 2 finanzas_schema.indicador_financiero modo: insertar al final

In [0]:
spark_df.write \
    .format("parquet") \
    .mode("append") \
    .partitionBy("cod_indicador") \
    .saveAsTable("finanzas_schema.indicador_financiero")

In [0]:
display(spark.sql("SELECT * FROM finanzas_schema.indicador_financiero"))

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,year,month,day,year_month,fecha_extraccion,cod_indicador
2024-12-04,995852.0,997511.0,993615.0,996948.0,996948.0,4450698.0,2024,12,4,2024-12,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-12-03,995406.0,997052.0,994180.0,995852.0,995852.0,2052002.0,2024,12,3,2024-12,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-12-02,991974.0,997222.0,987947.0,995406.0,995406.0,3362677.0,2024,12,2,2024-12,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-12-01,992652.0,995992.0,984977.0,991974.0,991974.0,846295.0,2024,12,1,2024-12,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-11-30,992427.0,996868.0,990345.0,992652.0,992652.0,2313050.0,2024,11,30,2024-11,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-11-29,994079.0,995848.0,991223.0,992427.0,992427.0,922440.0,2024,11,29,2024-11,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-11-28,990683.0,997390.0,989335.0,994079.0,994079.0,1159717.0,2024,11,28,2024-11,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-11-27,994844.0,995100.0,987895.0,990683.0,990683.0,667196.0,2024,11,27,2024-11,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-11-26,995305.0,1000345.0,990985.0,994844.0,994844.0,895951.0,2024,11,26,2024-11,2025-06-05T01:24:18.030+0000,EURUSD=X
2024-11-25,993739.0,996692.0,992103.0,995305.0,995305.0,501971.0,2024,11,25,2024-11,2025-06-05T01:24:18.030+0000,EURUSD=X


In [0]:
display(spark.sql("DESCRIBE EXTENDED finanzas_schema.indicador_financiero"))

col_name,data_type,comment
fecha,string,null
abrir,double,null
max,double,null
min,double,null
cerrar,double,null
cierre_ajustado,double,null
volumen,double,null
year,int,null
month,int,null
day,int,null


In [0]:
display(spark.sql("select count(*) from finanzas_schema.indicador_financiero"))

count(1)
365


### Tabla 3 - Cruce

In [0]:
%sql
DROP TABLE IF EXISTS finanzas_schema.tbl_indicadores;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS finanzas_schema.tbl_indicadores
USING parquet
PARTITIONED BY (cod_indicador)
COMMENT "Union de datos historicos finanzas_schema.indicador_financiero vs finanzas_schema.indicador_parametros"
AS
SELECT 
    x.cod_indicador ,
    x.fecha , 
    x.abrir ,
    x.max ,
    x.min ,
    x.cerrar ,
    x.cierre_ajustado ,
    x.volumen ,
---- **************** tabla parametros
    y.descripcion, 
    y.patron_clasificacion, 
    y.prioridad,
    y.activo,
    x.year ,
    x.month ,
    x.day ,
    x.year_month ,
    x.fecha_extraccion
FROM finanzas_schema.indicador_financiero AS x
LEFT JOIN finanzas_schema.indicador_parametros AS y
  ON x.cod_indicador = y.cod_indicador;

In [0]:
%sql
select count(*) from finanzas_schema.tbl_indicadores ;

count(1)
365


### tabla 4

In [0]:
%sql
DROP TABLE IF EXISTS finanzas_schema.ubicacion_indicador;

In [0]:
%sql


In [0]:
%sql
CREATE TABLE IF NOT EXISTS finanzas_schema.ubicacion_indicador (
  code STRING,
  name STRING NOT NULL,
  country STRING NOT NULL,
  lat DECIMAL(9,6) NOT NULL,
  lon DECIMAL(9,6) NOT NULL,
  region STRING NOT NULL,
  currency_pair STRING NOT NULL,
  descripcion STRING NOT NULL
)
USING DELTA
COMMENT 'Lista de indicadores financieros con sus descripciones';

In [0]:
# File location and type
file_location = "/FileStore/tables/bigdata_2025_1_2/registros_actualizado.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

code,name,country,latitude,longitude,continent,ticker,description
GBPUSD%3DX,Libra/Dólar,Reino Unido,51.5074,-0.1278,Europa,GBP/USD,Tipo de cambio entre la libra esterlina y el dólar estadounidense.
USDJPY%3DX,Dólar/Yen,Estados Unidos/Japón,35.6895,139.6917,Asia,USD/JPY,Tipo de cambio entre el dólar estadounidense y el yen japonés.
AUDUSD%3DX,Dólar australiano/Dólar,Australia,-33.8688,151.2093,Oceanía,AUD/USD,Tipo de cambio entre el dólar australiano y el dólar estadounidense.
USDCAD%3DX,Dólar/Dólar canadiense,Canadá,45.4215,-75.6972,América del Norte,USD/CAD,Tipo de cambio entre el dólar estadounidense y el dólar canadiense.
NZDUSD%3DX,Dólar neozelandés/Dólar,Nueva Zelanda,-41.2865,174.7762,Oceanía,NZD/USD,Tipo de cambio entre el dólar neozelandés y el dólar estadounidense.
USDCHF%3DX,Dólar/Franco suizo,Suiza,46.2044,6.1432,Europa,USD/CHF,Tipo de cambio entre el dólar estadounidense y el franco suizo.
USDBRL%3DX,Dólar/Real brasileño,Brasil,-15.7939,-47.8828,América del Sur,USD/BRL,Tipo de cambio entre el dólar estadounidense y el real brasileño.
USDMXN%3DX,Dólar/Peso mexicano,México,19.4326,-99.1332,América del Norte,USD/MXN,Tipo de cambio entre el dólar estadounidense y el peso mexicano.
USDCLP%3DX,Dólar/Peso chileno,Chile,-33.4489,-70.6693,América del Sur,USD/CLP,Tipo de cambio entre el dólar estadounidense y el peso chileno.
USDCNY%3DX,Dólar/Yuan chino,China,39.9042,116.4074,Asia,USD/CNY,Tipo de cambio entre el dólar estadounidense y el yuan chino.


In [0]:
%sql
DROP TABLE IF EXISTS finanzas_schema.indicadores_ubicacion;

In [0]:
%sql
DROP TABLE IF EXISTS finanzas_schema.ubicacion_indicador;

In [0]:
permanent_table_name = "finanzas_schema.ubicacion_indicador"
df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from finanzas_schema.indicadores_ubicacion limit 2;

code,name,country,latitude,longitude,continent,ticker,description
GBPUSD%3DX,Libra/Dólar,Reino Unido,51.5074,-0.1278,Europa,GBP/USD,Tipo de cambio entre la libra esterlina y el dólar estadounidense.
USDJPY%3DX,Dólar/Yen,Estados Unidos/Japón,35.6895,139.6917,Asia,USD/JPY,Tipo de cambio entre el dólar estadounidense y el yen japonés.


In [0]:
%sql
CREATE TABLE IF NOT EXISTS finanzas_schema.tbl_indicadores_ubicacion
USING parquet
PARTITIONED BY (cod_indicador)
COMMENT "Union de datos historicos finanzas_schema.indicador_financiero vs finanzas_schema.indicador_parametros"
AS
SELECT 
    x.cod_indicador ,
    x.fecha , 
    x.abrir ,
    x.max ,
    x.min ,
    x.cerrar ,
    x.cierre_ajustado ,
    x.volumen ,
---- **************** tabla parametros
    x.descripcion, 
    x.patron_clasificacion, 
    x.prioridad,
    x.activo,
    x.year ,
    x.month ,
    x.day ,
    x.year_month ,
    x.fecha_extraccion,
  ---- ****************** tabla 4
    y.name  ,
    y.country  ,
    y.latitude ,
    y.longitude ,
    y.continent  ,
    y.ticker  ,
    y.description  
FROM finanzas_schema.tbl_indicadores AS x
LEFT JOIN finanzas_schema.ubicacion_indicador AS y
  ON x.cod_indicador = y.code;

In [0]:
%sql
select * from finanzas_schema.tbl_indicadores_ubicacion limit 5;

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,descripcion,patron_clasificacion,prioridad,activo,year,month,day,year_month,fecha_extraccion,name,country,latitude,longitude,continent,ticker,description,cod_indicador
2024-12-04,995852.0,997511.0,993615.0,996948.0,996948.0,4450698.0,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true,2024,12,4,2024-12,2025-06-05T01:24:18.030+0000,Euro/Dólar,Unión Europea/Estados Unidos,52.52,13.405,Europa,EUR/USD,Tipo de cambio entre el euro y el dólar estadounidense.,EURUSD=X
2024-12-03,995406.0,997052.0,994180.0,995852.0,995852.0,2052002.0,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true,2024,12,3,2024-12,2025-06-05T01:24:18.030+0000,Euro/Dólar,Unión Europea/Estados Unidos,52.52,13.405,Europa,EUR/USD,Tipo de cambio entre el euro y el dólar estadounidense.,EURUSD=X
2024-12-02,991974.0,997222.0,987947.0,995406.0,995406.0,3362677.0,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true,2024,12,2,2024-12,2025-06-05T01:24:18.030+0000,Euro/Dólar,Unión Europea/Estados Unidos,52.52,13.405,Europa,EUR/USD,Tipo de cambio entre el euro y el dólar estadounidense.,EURUSD=X
2024-12-01,992652.0,995992.0,984977.0,991974.0,991974.0,846295.0,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true,2024,12,1,2024-12,2025-06-05T01:24:18.030+0000,Euro/Dólar,Unión Europea/Estados Unidos,52.52,13.405,Europa,EUR/USD,Tipo de cambio entre el euro y el dólar estadounidense.,EURUSD=X
2024-11-30,992427.0,996868.0,990345.0,992652.0,992652.0,2313050.0,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true,2024,11,30,2024-11,2025-06-05T01:24:18.030+0000,Euro/Dólar,Unión Europea/Estados Unidos,52.52,13.405,Europa,EUR/USD,Tipo de cambio entre el euro y el dólar estadounidense.,EURUSD=X
